In [ ]:
import json
import re
import logging

logger = logging.getLogger(__name__)

def parse_json_safely(text):
    logger.info(f"Attempting to parse JSON from: {text}")
    
    # First, try to parse the entire text as JSON
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass  # If it fails, continue to more robust parsing
    
    # Try to extract JSON-like structure
    json_match = re.search(r'\{(?:[^{}]|(?R))*\}', text)
    if json_match:
        json_str = json_match.group(0)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            pass  # If it fails, continue to manual parsing
    
    # If JSON extraction fails, try to manually parse key-value pairs
    manual_parse = {}
    key_value_pattern = r'"?([\w_]+)"?\s*:\s*("(?:\\.|[^"\\])*"|\d+|true|false|null)'
    matches = re.finditer(key_value_pattern, text)
    
    for match in matches:
        key, value = match.groups()
        try:
            # Attempt to parse the value
            parsed_value = json.loads(value)
        except json.JSONDecodeError:
            # If parsing fails, use the string value
            parsed_value = value.strip('"')
        manual_parse[key] = parsed_value
    
    if manual_parse:
        logger.info(f"Manually parsed JSON: {manual_parse}")
        return manual_parse
    
    logger.warning("Failed to parse any JSON-like structure")
    return None

# Example usage and testing
if __name__ == "__main__":
    test_cases = [
        '{"year_s": "2023", "ticker_s": "AAPL", "quarter_s": "Q2"}',
        'Here is the JSON: {"year_s": "2023", "ticker_s": "AAPL", "quarter_s": "Q2"}',
        'The JSON is: {year_s: "2023", ticker_s: "AAPL", quarter_s: "Q2"}',
        'year_s: "2023", ticker_s: "AAPL", quarter_s: "Q2"',
        'Invalid JSON {year_s: 2023, ticker_s: AAPL, quarter_s: Q2}',
    ]

    for i, case in enumerate(test_cases, 1):
        result = parse_json_safely(case)
        print(f"Test case {i}:")
        print(f"Input: {case}")
        print(f"Result: {result}")
        print()